In [49]:
from os.path import join, basename
from time import sleep

from tqdm.auto import tqdm
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, mapping
import pystac_client
from matplotlib import pyplot as plt

In [11]:
df = pd.read_csv('/home/ahassan/un-sandstorm/data/2022.csv', parse_dates=['date'], date_format='%Y-%b-%d')
geometry = [Point(lon, lat) for lon, lat in zip(df['lon'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
gdf.head()

,date,lat,lon,geometry
0,2022-01-06,29.5351,58.7577,POINT (58.75770 29.53510)
1,2022-01-11,36.5726,62.4626,POINT (62.46260 36.57260)
2,2022-01-14,37.5128,65.0667,POINT (65.06670 37.51280)
3,2022-01-21,29.4008,58.8920,POINT (58.89200 29.40080)
4,2022-01-22,28.2458,63.2680,POINT (63.26800 28.24580)


In [29]:
URL = 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/'
catalog = pystac_client.Client.open(URL)
collections = ['VNP09GA.v002']

In [48]:
stac_items = {}
with tqdm(gdf.itertuples(index=False), total=len(gdf)) as bar:
    for row in bar:
        search = catalog.search(
            intersects=mapping(row.geometry),
            collections=collections,
            datetime=str(row.date.date()),
        )
        items = search.item_collection()
        for item in items:
            if item.id in stac_items:
                continue
            stac_items[item.id] = item
        sleep(0.5)
len(stac_items)

  0%|          | 0/165 [00:00<?, ?it/s]

163

In [55]:
download_dir = '/home/ahassan/un-sandstorm/data/downloaded/2022'
src_uri = item.assets['data'].href
filename = basename(src_uri)
dst_path = join(download_dir, filename)
print(dst_path)

/home/ahassan/un-sandstorm/data/downloaded/2022/VNP09GA.A2022270.h21v07.002.2023058081633.h5


In [57]:
from rastervision.pipeline.file_system import HttpFileSystem

In [58]:
bearer_token = ''

In [59]:
HttpFileSystem.copy_from(src_uri, dst_path, headers={'Authorization': f'Bearer {bearer_token}'})

In [61]:
!rm {dst_path}